In [8]:
from pathlib import Path

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI


PASTA_ARQUIVOS = Path('C:/Users/avsoares/OneDrive/Documentos/GitHub/courses/Langchain/projeto/teste.ipynb').parent / 'arquivos'
MODEL_NAME = 'gpt-3.5-turbo-0125'


def importacao_documentos():
    documentos = []
    for arquivo in PASTA_ARQUIVOS.glob('*.pdf'):
        loader = PyPDFLoader(str(arquivo))
        documentos_arquivo = loader.load()
        documentos.extend(documentos_arquivo)
    return documentos

def split_de_documentos(documentos):
    recur_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=250,
        separators=["/n\n", "\n", ".", " ", ""]
    )
    documentos = recur_splitter.split_documents(documentos)

    for i, doc in enumerate(documentos):
        doc.metadata['source'] = doc.metadata['source'].split('/')[-1]
        doc.metadata['doc_id'] = i
    return documentos

def cria_vector_store(documentos):
    embedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(
        documents=documentos,
        embedding=embedding_model
    )
    return vector_store

def cria_chain_conversa(vector_store):
    chat = ChatOpenAI(model=MODEL_NAME)
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key='answer'
        )
    retriever = vector_store.as_retriever()
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retriever,
        return_source_documents=True,
        verbose=True
    )
    return chat_chain

In [9]:
documentos = importacao_documentos()
documentos = split_de_documentos(documentos)
vector_store = cria_vector_store(documentos)
chain = cria_chain_conversa(vector_store)

In [10]:
chain.invoke({'question': 'O que é a openAI?'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
•GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que aceita
entradas de texto ou imagem e produz texto. Ele é capaz de resolver problemas complexos com
maior precisão do que os modelos anteriores da OpenAI.
•DALL-E: Este sistema de IA pode criar imagens e arte realistas a partir de descrições textuais. O
DALL ·E 3, em particular, suporta a capacidade de 

{'question': 'O que é a openAI?',
 'chat_history': [HumanMessage(content='O que é a openAI?'),
  AIMessage(content='A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade. Ela é responsável por criar diversos modelos de IA, como o GPT-4, DALL-E, TTS, e Whisper, que possuem funcionalidades específicas, como linguagem de grande escala, criação de imagens realistas a partir de descrições textuais, transformação de texto em fala, reconhecimento de fala e tradução de idiomas.')],
 'answer': 'A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade. Ela é responsável por criar diversos modelos de IA, como o GPT-4, DALL-E, TTS, e Whisper, que possuem funcionalidades específicas, como linguagem de grande escala, criação de imagens realistas a partir de descrições textuais, transformação de texto em fala, reconhecimento de fala e tradução de idiomas.',
 'source_documents': [Document(page_content='Explorando a 